In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

<font size="S" color="red">ch15 데이터베이스연동</font>
# 1절 SQLite 데이터 베이스 연동
- sqlite 데이터베이스는 별도의 DBMS 없이 SQL 을 이용하여 DB 엑세스 가능한 디스크 기반 DB
- sqlite는 프로토타입(시제품) 용도
- 프로젝트 단계: 분석- 설계 -구현- 테스트- 고객에게 배포- 유지보수
                 SQlite      oracle/Mysql,MariaDB, mongoDB
- c 라이브러리


# 1.1 SQLite 패키지 load

In [1]:
import sqlite3
sqlite3.sqlite_version

'3.40.1'

In [2]:
import pandas as pd
pd.__version__

'1.5.3'

# 1.2 데이터베이스연결

In [3]:
# DB연결
conn = sqlite3.connect('data/ch15_example.db')
conn

In [4]:
#커서객체(sql문 실행 -> 결과조회) 생성
cursor = conn.cursor()
cursor

In [5]:
cursor.execute('''
    CREATE TABLE MEMBER(
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
        )
''')

In [13]:
cursor.execute('''
    INSERT INTO MEMBER VALUES('최길동',20,'h@h.com')
''')
print('수행 결과 행수:',cursor.rowcount)

수행 결과 행수: 1


In [14]:
sql = "INSERT INTO MEMBER VALUES('김준서',30,'k@h.com')"
print(sql)
cursor.execute(sql)
print('수행결과행수:',cursor.rowcount)

INSERT INTO MEMBER VALUES('김준서',30,'k@h.com')
수행결과행수: 1


In [36]:
conn.commit() #(반) conn,rollback DML에서만 가능

### SELECT 전송


In [29]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER
''')

In [30]:
#SELECT 전송
#   select 문 실행결과를 받는 함수
#   cursor.fetchone(): 결과를 한행씩 받을 때 (튜플 list)
#   cursor.fetchmany(n): 결과를 n행 받을 때(튜플 list)
#   cursor.description: header 내용을 포함한 내용들(ㅣist)

#INSERT, UPDATE, DELETE 전송: cursor,rowcount
print(cursor.fetchall())

[('홍길동', 21, 'h@h.com'), ('김준서', 31, 'k@h.com'), ('최길동', 21, 'h@h.com'), ('김준서', 31, 'k@h.com')]


In [33]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by NEXTAGE
''')
members = cursor.fetchone()
members

('홍길동', 21, 'h@h.com')

In [34]:
cursor.fetchall()

[('최길동', 21, 'h@h.com'), ('김준서', 31, 'k@h.com'), ('김준서', 31, 'k@h.com')]

In [35]:
cursor.execute('''
    SELECT NAME, AGE+1 NEXTAGE, EMAIL FROM MEMBER order by NEXTAGE
''')
members = []
while True:
    member = cursor.fetchone()
    if member is None:
        break
    print(member)
    members.append({'name':member[0], 'age':member[1], 'email':member[2]})
import pandas as pd
pd.DataFrame(members)

('홍길동', 21, 'h@h.com')
('최길동', 21, 'h@h.com')
('김준서', 31, 'k@h.com')
('김준서', 31, 'k@h.com')


,name,age,email
0,홍길동,21,h@h.com
1,최길동,21,h@h.com
2,김준서,31,k@h.com
3,김준서,31,k@h.com


In [27]:
#select 문을 수행한 필드정보
cursor.description

(('NAME', None, None, None, None, None, None),
 ('NEXTAGE', None, None, None, None, None, None),
 ('EMAIL', None, None, None, None, None, None))

In [37]:
[description[0] for description in cursor.description]

['NAME', 'NEXTAGE', 'EMAIL']

## 1.3 SQL 구문에 파라미터 사용하기 
- qmark(DB에 따라 불가한 경우 있음)
- named(추천)

In [38]:
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','김준서')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('김준서', 30, 'k@h.com'), ('김준서', 30, 'k@h.com')]

In [41]:
# qmark 방법 이용: 파라미터 사용하기
conn = sqlite3.connect('data/ch15_example.db')
cursor = conn.cursor()
sql = "SELECT * FROM MEMBER WHERE NAME IN (?,?)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른 이름은?')
cursor.execute(sql,(name1,name2))
cursor.fetchall()

검색할 이름은?홍길동
검색할 다른 이름은?김준서


[('홍길동', 20, 'h@h.com'), ('김준서', 30, 'k@h.com'), ('김준서', 30, 'k@h.com')]

In [42]:
#파라미터 사용하기 : named 방법 이용
sql = "SELECT * FROM MEMBER WHERE NAME IN (:name1,:name2)"
name1 = input('검색할 이름은?')
name2 = input('검색할 다른 이름은?')
cursor.execute(sql,(name1,name2))
cursor.fetchall()

검색할 이름은?김준서
검색할 다른 이름은?김길동


[('김준서', 30, 'k@h.com'), ('김준서', 30, 'k@h.com')]

In [44]:
#named 방법으로 데이터 입력하기 
try:
    name = input('입력할 이름은?')
    age = int(input('입력할 나이는?'))
except ValueError:
    print('유효하지 않은 나이를 입력하시면 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은?')
cursor.execute('INSERT INTO MEMBER VALUES (:name,:age,:email)',
                        {'name':name, 'age':age, 'email':email})
conn.commit()
if cursor.rowcount == 1:
    print('입력성공')
else:
    print('입력실패')
    
                            

입력할 이름은?정다온
입력할 나이는?21
입력할 메일은?daon@naver.com
입력성공


# 2절 오라클 데이터 베이스 연결
- pip install cx_oracle

In [45]:
import cx_Oracle
#conn 열어오는 방법 1
conn = cx_Oracle.connect('scott','tiger','localhost:1521/xe')
conn

<cx_Oracle.Connection to scott@localhost:1521/xe>

In [49]:
# conn 얻어오는 방법2
oracle_dsn = cx_Oracle.makedsn(host='localhost',port=1521, sid="xe")
conn = cx_Oracle.connect("scott", "tiger", dsn=oracle_dsn)
conn

<cx_Oracle.Connection to scott@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [50]:
cursor = conn.cursor()
cursor.execute('SELECT EMPNO NO, ENAME, JOB, MGR,SAL,COMM FROM EMP')
emps = cursor.fetchall()
for emp in emps:
    print(emp)

(7369, 'SMITH', 'CLERK', 7902, 800.0, None)
(7499, 'ALLEN', 'SALESMAN', 7698, 1600.0, 300.0)
(7521, 'WARD', 'SALESMAN', 7698, 1250.0, 500.0)
(7566, 'JONES', 'MANAGER', 7839, 2975.0, None)
(7654, 'MARTIN', 'SALESMAN', 7698, 1250.0, 1400.0)
(7698, 'BLAKE', 'MANAGER', 7839, 2850.0, None)
(7782, 'CLARK', 'MANAGER', 7839, 2450.0, None)
(7788, 'SCOTT', 'ANALYST', 7566, 3000.0, None)
(7839, 'KING', 'PRESIDENT', None, 5000.0, None)
(7844, 'TURNER', 'SALESMAN', 7698, 1500.0, 0.0)
(7876, 'ADAMS', 'CLERK', 7788, 1100.0, None)
(7900, 'JAMES', 'CLERK', 7698, 950.0, None)
(7902, 'FORD', 'ANALYST', 7566, 3000.0, None)
(7934, 'MILLER', 'CLERK', 7782, 1300.0, None)
